In [11]:
!pip install -e generator/generator_scripts

Obtaining file:///home/ec2-user/SageMaker/kolmogorov/generator/generator_scripts
     |████████████████████████████████| 348.0 MB 5.1 kB/s  eta 0:00:01   |███▏                            | 34.5 MB 3.1 MB/s eta 0:01:42     |████████████████████▎           | 221.1 MB 61.0 MB/s eta 0:00:03     |█████████████████████           | 229.3 MB 61.0 MB/s eta 0:00:02     |████████████████████████▏       | 262.2 MB 83.0 MB/s eta 0:00:02
  Using cached opencv_python-4.3.0.36-cp36-cp36m-manylinux2014_x86_64.whl (43.7 MB)
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.2.0.32
    Uninstalling opencv-python-4.2.0.32:
      Successfully uninstalled opencv-python-4.2.0.32
  Running setup.py develop for generator-scripts
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/tensorflow_p36/bin/python -m pip install --upgrade pip' command.


In [16]:
!distort_dataset ../ssa-data/updated-orientation/not_distorted "**/*.png" ../ssa-data/updated-orientation/more_distorted 8 0.351 100 none 32

Attempting to distort images matching pattern "**/*.png" in directory: ../ssa-data/updated-orientation/not_distorted. Output directory: ../ssa-data/updated-orientation/more_distorted
Failed to apply distortion to images -- Unable to allocate 516. MiB for an array with shape (8224, 8224) and data type float64
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/bin/distort_dataset", line 7, in <module>
    exec(compile(f.read(), __file__, 'exec'))
  File "/home/ec2-user/SageMaker/kolmogorov/generator/generator_scripts/bin/distort_dataset", line 44, in <module>
    raise e
  File "/home/ec2-user/SageMaker/kolmogorov/generator/generator_scripts/bin/distort_dataset", line 41, in <module>
    atmospheric_distort_directory(directory_path, file_glob_matcher, output_directory, aperture_size, fried_param, outer_scale, random_seed, stencil_length_factor)
  File "/home/ec2-user/SageMaker/kolmogorov/generator/generator_scripts/generator_scripts/distortion.py", li

In [13]:
!pwd

/home/ec2-user/SageMaker/kolmogorov


In [ ]:
!distort_dataset ../ssa-data/updated-orientation/not_distorted "*/*.png" ../ssa-data/updated-orientation/less_distorted 8 1.6 100 none 32

In [42]:
!generic_dataset ../ssa-data ../structured-generic

Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/bin/generic_dataset", line 7, in <module>
    exec(compile(f.read(), __file__, 'exec'))
  File "/home/ec2-user/SageMaker/kolmogorov/generator/generator_scripts/bin/generic_dataset", line 45, in <module>
    for sat in os.listdir(SUBDIR(img_dir)):
FileNotFoundError: [Errno 2] No such file or directory: '../ssa-data/not_distorted/test'


In [1]:
import os
import numpy as np
import tensorflow as tf

In [2]:
tf.compat.v1.disable_eager_execution()

In [15]:
@tf.function
def path2img(path):
    img = tf.io.read_file(path)
    img = tf.io.decode_image(img)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return img

In [16]:
@tf.function
def tf_deg2rad(deg):
    pi_on_180 = 0.017453292519943295
    return deg * pi_on_180

In [17]:
@tf.function
def path2orientation(path):
    orientation_str = tf.strings.split(path, '_')[2]
    orientation = tf.strings.to_number(tf.strings.split(orientation_str, ','))
    orientation = tf_deg2rad(orientation)
    orientation = tf.concat([tf.math.sin(orientation), tf.math.cos(orientation)], 0)
    return orientation

In [18]:
@tf.function
def path2label(path):
    parts = tf.strings.split(path, os.path.sep)
    return tf.cast(classes == parts[-2], tf.float32)

In [4]:
directory = '/home/ec2-user/SageMaker/kolmogorov/model/validation'
batch_size = 16
classes = tf.convert_to_tensor(sorted(os.listdir(directory)))
files_ds = tf.data.Dataset.list_files(os.path.join(directory,'*/*.png'))

In [6]:
test_ds = files_ds.map(lambda x: path2img(x))

In [83]:
orientation_ds = files_ds.map(
    lambda x: tf.py_function(
        path2orientation, [x], [tf.float32]
    ),
    num_parallel_calls=tf.data.experimental.AUTOTUNE
)


labels_ds = files_ds.map(
    lambda x: tf.py_function(
        path2label, [x, classes], [tf.float32]
    ),
    num_parallel_calls=tf.data.experimental.AUTOTUNE
)

img_ds = files_ds.map(
    lambda x: tf.py_function(
        path2img, [x], [tf.float32]
    ),
    num_parallel_calls=tf.data.experimental.AUTOTUNE
)

outputs_ds = tf.data.Dataset.zip((labels_ds, orientation_ds))
ds = tf.data.Dataset.zip((img_ds, outputs_ds))
#ds = ds.map(lambda x,y: [tf.convert_to_tensor(x[0]), [tf.convert_to_tensor(y[0]), tf.convert_to_tensor(y[1])]])
ds = ds.batch(batch_size)
#ds = ds.shuffle(files_ds.cardinality().numpy(), reshuffle_each_iteration=True)
#ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)